# 任务 1b：使用包含上下文的提示词执行文本生成

在此笔记本中，您将学习如何向对客户支持工程师提供的客户服务质量不满意的客户生成回复电子邮件。您将提供不满意的客户发来的实际电子邮件内容，从而为模型提供更多上下文。

您将借助 PromptTemplates 来增加复杂性，以利用 LangChain 框架处理类似的使用案例。PromptTemplates 使您可以创建通用 Shell，这些 Shell 之后可填充信息并根据不同的场景获得模型输出。

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) 是一个框架，用于开发由语言模型提供支持的应用程序。此框架的关键方面是，我们可通过将各种组件链接在一起来增强大型语言模型，从而创建高级使用案例。

由于提示词中有更多上下文，此笔记本中所生成内容的质量和相关性都比之前通过零样本提示词所生成的内容要好得多。此笔记本中使用的提示词会创建一个自定义 LangChain 提示词模板，用于向文本生成请求添加上下文。

#### 场景
您是 AnyCompany 的客户服务经理 Bob，您的部分客户对客户服务不满意，并对客户支持工程师提供的服务给出了负面反馈。现在，您要在回复中为糟糕的服务向那些客户谦逊地道歉，并重获信任。您需要在大型语言模型 (LLM) 的帮助下生成大量电子邮件，这些电子邮件要人性化，并根据客户在之前电子邮件通信中展现的情绪进行个性化定制。

在此场景中，您可以利用 LangChain 的 PromptTemplates 的强大功能来创建一个通用 Shell，用于根据客户之前的电子邮件生成个性化的回复电子邮件。PromptTemplate 将整合客户的原始电子邮件内容，使 LLM 能够了解上下文和客户情绪，然后生成相关的自定义响应。

## 任务 1b.1：环境设置

在此任务中，您将设置环境。

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))


bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## 任务 1b.2：调用 Bedrock LLM 模型

在此任务中，您将通过 LLM 创建一个 Bedrock 类实例。这需要一个 `model_id`，它是 Amazon Bedrock 中的模型的 Amazon Resource Name (ARN)。

（可选）您可以传递之前创建的 boto3 客户端以及一些 `model_kwargs`，后者可以包含参数，例如 `temperature`、`top_p`、`max_token_count` 或 `stop_sequences`（可在 Amazon Bedrock 控制台中了解有关参数的更多信息）。

有关 Amazon Bedrock 中可用的文本生成模型 ID，请参阅 [文档](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html)。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i>**注意**：不同的模型支持不同的 `model_kwargs`。

In [ ]:
# Model configuration
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser

model_id = "meta.llama3-8b-instruct-v1:0"
model_kwargs =  { 
        "max_gen_len": 512,
        "temperature": 0,
        "top_p": 1,
}

# LangChain class for chat
chat_model = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

## 任务 1b.3：创建 LangChain 自定义提示词模板

在此任务中，您将创建一个提示词模板，用于在每次运行时传递不同的输入变量。当您必须使用可能从数据库中获取的不同输入变量生成内容时，这非常有用。

在上一个任务中，我们对提示词进行了硬编码。这可能适合多位客户向您提供了相似的负面反馈的情况，而现在，您想分别回复每位客户的电子邮件以向他们道歉，并且希望回复实现个性化。在下面的单元中，您将探索如何创建 `PromptTemplate` 来实现此模式。

In [ ]:
# Create a prompt template that has multiple input variables
from langchain.prompts import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} at AnyCompany to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob Smith", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [ ]:
# get number of tokens
num_tokens = chat_model.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i>**注意**：您可以放心地忽略这些警告，并继续执行下一个单元。

In [ ]:
#invoke
response = chat_model.invoke(prompt)

In [ ]:
# Configure a Chain to parse output
chain = StrOutputParser()
formatted_response=chain.invoke(response)
print(formatted_response)

您已了解了在不提供任何上下文的情况下调用 LLM 可能不会生成期望的结果。通过添加上下文并进一步使用提示词模板来限制 LLM 的输出，您可以成功获得期望的输出。

### 自行尝试
- 将提示词更改为您的特定使用案例，并评估不同模型的输出。
- 尝试不同的 Token 长度，了解服务的延迟和响应能力。
- 应用不同的提示词工程原则，获得更好的输出。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续执行**任务 2**。